# Color Scheme Generator

The purpose of this notebook is to generate large sets of attractive and accessible color schemes based on Mathew Strom's work in [*How to pick the least wrong colors*](https://matthewstrom.com/writing/how-to-pick-the-least-wrong-colors).

In [1]:
# const chroma = require("chroma-js");
import random
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000

In [ ]:
# const targetColors = [
#     "#9966FF",
#     "#0055BC",
#     "#00A1C2",
#     "#ED6804",
#     "#B3063D"
# ];
targetColors = [
    "#9966FF",
    "#0055BC",
    "#00A1C2",
    "#ED6804",
    "#B3063D"
]

In [ ]:
# // random from array
# const randomFromArray = (array) => {
#     return array[Math.floor(Math.random() * array.length)];
# };

def randomFromArray(array):
    """
    Selects a random element from a given array (list).

    This function takes an array (list in Python terminology) and returns
    a random element from it. The function does not modify the original array.

    Parameters:
    array (list): A list from which a random element is to be selected. 
                  The list should not be empty.

    Returns:
    element: A randomly selected element from the input list. The type of
             the element depends on the content of the list.

    Raises:
    ValueError: If the input array is empty.

    Example:
        >>> my_list = [1, 2, 3, 4, 5]
        >>> print(randomFromArray(my_list))
        3  # This is an example output, actual output will be random.
    """

    if not array:
        raise ValueError("Input array is empty.")

    return random.choice(array)

In [ ]:
# // generate a random color
# const randomColor = () => {
#     const color = chroma.random();
#     return color;
# };

def randomColor():
    """
    Generate a random hexadecimal color.

    This function produces a random color in hexadecimal format. The color
    is represented as a string in the format '#RRGGBB', where RR, GG, and BB
    are two hexadecimal digits representing the red, green, and blue color values.

    Returns:
        str: A string representing the hexadecimal color.

    Example:
        >>> random_color = randomColor()
        >>> print(random_color)
        #92a8d1
    """
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

In [ ]:
# // measures the distance between two colors
# const distance = (color1, color2) => chroma.deltaE(color1, color2);

def distance(color1_hex, color2_hex):
    """
    Calculates the CIE ΔE* (Delta E) color difference between two colors.

    The function takes two color values in hexadecimal format and calculates
    their perceptual color difference using the CIE2000 ΔE* formula.
    The ΔE* value ranges from 0 to 100, where 0 means the colors are identical,
    and 100 indicates maximum perceptual difference.

    Parameters:
    color1_hex (str): The first color in hexadecimal format (e.g., "#FFFFFF").
    color2_hex (str): The second color in hexadecimal format (e.g., "#000000").

    Returns:
    float: The ΔE* value representing the difference between the two colors.

    Example:
        >>> color1 = "#FF5733"
        >>> color2 = "#4CAF50"
        >>> print(distance(color1, color2))
        57.82  # Example output, actual output will vary.

    Note:
    This function requires the 'colormath' library.
    """

    # Convert hex colors to sRGB
    color1_rgb = sRGBColor.new_from_rgb_hex(color1_hex)
    color2_rgb = sRGBColor.new_from_rgb_hex(color2_hex)

    # Convert sRGB colors to Lab color space
    color1_lab = convert_color(color1_rgb, LabColor)
    color2_lab = convert_color(color2_rgb, LabColor)

    # Calculate and return the Delta E value
    return delta_e_cie2000(color1_lab, color2_lab)

In [ ]:
# const getClosestColor = (color, colorArray) => {
#     const distances = colorArray.map((c) => distance(color, c));
#     const minIndex = distances.indexOf(Math.min(...distances));
#     return colorArray[minIndex];
# };

def getClosestColor(color, colorArray):
    """
    Finds the color in 'colorArray' that is closest to the given 'color'.

    This function computes the CIE ΔE* color difference between the given 'color' and each color in
    'colorArray', and returns the color from 'colorArray' that has the smallest ΔE* value to 'color'.

    Parameters:
    color (str): The reference color in hexadecimal format (e.g., "#FFFFFF").
    colorArray (list): A list of colors in hexadecimal format (e.g., ["#000000", "#FF0000", ...]).

    Returns:
    str: The hexadecimal color code from 'colorArray' closest to 'color'.

    Example:
        >>> color = "#FF5733"
        >>> colorArray = ["#4CAF50", "#FF0000", "#00FF00"]
        >>> print(getClosestColor(color, colorArray))
        #FF0000  # Example output, actual output will vary.

    Note:
    This function requires the 'distance' function previously defined.
    """

    # Initialize minimum distance and closest color
    min_distance = None
    closest_color = None

    # Iterate over each color in the array to find the closest one
    for color_option in colorArray:
        current_distance = distance(color, color_option)
        if min_distance is None or current_distance < min_distance:
            min_distance = current_distance
            closest_color = color_option

    return closest_color

In [ ]:
# // array of distances between all points in a color array
# const distances = (colorArray, visionSpace = "Normal") => {
#     const distances = [];
#     const convertedColors = colorArray.map((c) =>
#         brettelFunctions[visionSpace](c.rgb())
#     );
#     for (let i = 0; i < colorArray.length; i++) {
#         for (let j = i + 1; j < colorArray.length; j++) {
#             distances.push(distance(convertedColors[i], convertedColors[j]));
#         }
#     }
#     return distances;
# };

def distances(colorArray, visionSpace="Normal"):
    """
    Computes an array of distances between all pairs of colors in a color array.

    This function processes each color in 'colorArray' according to a specified
    'visionSpace', then calculates and returns the distances between every unique
    pair of processed colors using the CIE ΔE* color difference.

    Parameters:
    colorArray (list): A list of colors in hexadecimal format.
    visionSpace (str): The type of vision space to use for color processing. Defaults to "Normal".

    Returns:
    list: A list containing the ΔE* values between every unique pair of colors in 'colorArray'.

    Example:
        >>> colorArray = ["#FF5733", "#4CAF50", "#FFC107"]
        >>> print(distances(colorArray))
        [47.85, 52.16, 18.95]  # Example output, actual output will vary.

    Note:
    This function requires a vision space processing function equivalent to 'brettelFunctions' in JavaScript.
    It also requires the 'distance' function previously defined.
    """

    processed_colors = [process_color(c, visionSpace) for c in colorArray]  # Assuming process_color function exists
    distance_values = []

    for i in range(len(processed_colors)):
        for j in range(i + 1, len(processed_colors)):
            dist = distance(processed_colors[i], processed_colors[j])
            distance_values.append(dist)

    return distance_values

In [ ]:
# // get average of interger array
# const average = (array) => array.reduce((a, b) => a + b) / array.length;

def average(array):
    """
    Calculate the average of a list of integers.

    This function takes a list of integers and returns the average value. 
    If the list is empty, it returns 0. It uses a straightforward method of 
    summing all the integers in the list and then dividing by the number of 
    elements in the list.

    Parameters:
    array (list of int): The list of integers to calculate the average of.

    Returns:
    float: The average value of the integers in the list.
    """
    if not array:  # Check if the list is empty
        return 0
    return sum(array) / len(array)

In [ ]:
# // get the distance between the highest and lowest values in an array
# const range = (array) => {
#     const sorted = array.sort((a, b) => a - b);
#     return sorted[sorted.length - 1] - sorted[0];
# };

def range(array):
    """
    Calculate the distance between the highest and lowest values in a list.

    This function sorts the list of integers and calculates the difference 
    between the highest and lowest values. It returns this distance as an integer.
    If the list is empty or contains only one element, the function returns 0.

    Parameters:
    array (list of int): The list of integers to calculate the range distance of.

    Returns:
    int: The distance between the highest and lowest values in the list.
    """
    if len(array) < 2:  # Check if the list is empty or has only one element
        return 0
    sorted_array = sorted(array)
    return sorted_array[-1] - sorted_array[0]

In [ ]:
# // produces a color a small random distance away from the given color
# const randomNearbyColor = (color) => {
#     const channelToChange = randomFromArray([0, 1, 2]);
#     const oldVal = color.gl()[channelToChange];
#     let newVal = oldVal + Math.random() * 0.1 - 0.05;
#     if (newVal > 1) {
#         newVal = 1;
#     } else if (newVal < 0) {
#         newVal = 0;
#     }
#     return color.set(`rgb.${"rgb"[channelToChange]}`, newVal * 255);
# };

def randomNearbyColor(color):
    """
    Produces a color a small random distance away from the given color.

    This function takes a color in the form of a (r, g, b) tuple, where each
    component is an integer in the range 0-255. It then modifies one of the color
    channels (red, green, or blue) by a small random amount, without exceeding 
    the 0-255 range for any channel. The modified color is returned as a new tuple.

    Parameters:
    color (tuple of int): The original color in (r, g, b) format.

    Returns:
    tuple of int: The new color in (r, g, b) format, slightly different from the original.
    """
    channel_to_change = random.choice([0, 1, 2])  # Choose a random channel (r, g, or b)
    old_val = color[channel_to_change] / 255  # Normalize to range 0-1
    new_val = old_val + random.uniform(-0.05, 0.05)  # Modify by a small random amount

    # Clamp values to the range 0-1
    new_val = min(max(new_val, 0), 1)

    # Convert back to 0-255 range and update the color
    new_color = list(color)
    new_color[channel_to_change] = int(new_val * 255)

    return tuple(new_color)

In [ ]:
# // average of distances between array of colors and stripe colors
# const averageDistanceFromtargetColors = (colors) => {
#     const distances = colors.map((c) =>
#         distance(c, getClosestColor(c, targetColors))
#     );
#     return average(distances);
# };

def averageDistanceFromtargetColors(colors):
    global targetColors
    """
    Calculate the average of distances between an array of colors and the closest colors
    in a target color array.

    This function takes an array of colors and calculates the distance of each color
    to its closest color in a target color array. It then computes the average of these
    distances. The function assumes the existence of 'distance', 'getClosestColor', and
    'average' functions.

    Parameters:
    colors (list of tuples): The array of colors, each represented as an (r, g, b) tuple.
    target_colors (list of tuples): The target array of colors, each represented as an (r, g, b) tuple.

    Returns:
    float: The average distance between the colors in the array and their closest colors in the target array.
    """
    # Calculate distances from each color to its closest target color
    distances = [distance(c, get_closest_color(c, targetColors)) for c in colors]

    # Compute the average distance
    return average(distances)

In [ ]:
# // convert a linear rgb value to sRGB
# const linearRGB_from_sRGB = (v) => {
#     var fv = v / 255.0;
#     if (fv < 0.04045) return fv / 12.92;
#     return Math.pow((fv + 0.055) / 1.055, 2.4);
# }

def linearRGB_from_sRGB(value):
    """
    Convert a linear RGB value to sRGB.

    This function takes a single sRGB value (assumed to be in the range 0-255)
    and converts it to a linear RGB value. The conversion takes into account the
    nonlinear relationship between the two color spaces. For values below a certain
    threshold, a linear transformation is applied. For values above this threshold, 
    a nonlinear transformation is applied.

    Parameters:
    value (int): The sRGB value to be converted, in the range 0-255.

    Returns:
    float: The corresponding linear RGB value.
    """
    fv = value / 255.0
    if fv < 0.04045:
        return fv / 12.92
    return ((fv + 0.055) / 1.055) ** 2.4

In [ ]:
# const sRGB_from_linearRGB = (v) => {
#     if (v <= 0) return 0;
#     if (v >= 1) return 255;
#     if (v < 0.0031308) return 0.5 + v * 12.92 * 255;
#     return 0 + 255 * (Math.pow(v, 1.0 / 2.4) * 1.055 - 0.055);
# }

def sRGB_from_linearRGB(value):
    """
    Convert a linear RGB value to sRGB.

    This function takes a single linear RGB value (assumed to be in the range 0-1)
    and converts it to an sRGB value. The conversion process involves either a linear
    or a nonlinear transformation based on the value of the input. The function 
    ensures that the output is clamped to the range 0-255.

    Parameters:
    value (float): The linear RGB value to be converted, in the range 0-1.

    Returns:
    int: The corresponding sRGB value, in the range 0-255.
    """
    if value <= 0:
        return 0
    if value >= 1:
        return 255
    if value < 0.0031308:
        return int(0.5 + value * 12.92 * 255)
    return int(0 + 255 * ((value ** (1.0 / 2.4)) * 1.055 - 0.055))

In [ ]:
// Bretel et al method for simulating color vision deficiency
// Adapted from https://github.com/MaPePeR/jsColorblindSimulator
// In turn adapted from libDaltonLens https://daltonlens.org (public domain) 

const brettelFunctions = {
    Normal: function (v) {
        return v;
    },
    Protanopia: function (v) {
        return brettel(v, "protan", 1.0);
    },
    Protanomaly: function (v) {
        return brettel(v, "protan", 0.6);
    },
    Deuteranopia: function (v) {
        return brettel(v, "deutan", 1.0);
    },
    Deuteranomaly: function (v) {
        return brettel(v, "deutan", 0.6);
    },
    Tritanopia: function (v) {
        return brettel(v, "tritan", 1.0);
    },
    Tritanomaly: function (v) {
        return brettel(v, "tritan", 0.6);
    },
    Achromatopsia: function (v) {
        return monochrome_with_severity(v, 1.0);
    },
    Achromatomaly: function (v) {
        return monochrome_with_severity(v, 0.6);
    },
};

var sRGB_to_linearRGB_Lookup = Array(256);
(function () {
    var i;
    for (i = 0; i < 256; i++) {
        sRGB_to_linearRGB_Lookup[i] = linearRGB_from_sRGB(i);
    }
})();

brettel_params = {
    protan: {
        rgbCvdFromRgb_1: [
            0.1451, 1.20165, -0.34675, 0.10447, 0.85316, 0.04237, 0.00429,
            -0.00603, 1.00174,
        ],
        rgbCvdFromRgb_2: [
            0.14115, 1.16782, -0.30897, 0.10495, 0.8573, 0.03776, 0.00431,
            -0.00586, 1.00155,
        ],
        separationPlaneNormal: [0.00048, 0.00416, -0.00464],
    },

    deutan: {
        rgbCvdFromRgb_1: [
            0.36198, 0.86755, -0.22953, 0.26099, 0.64512, 0.09389, -0.01975,
            0.02686, 0.99289,
        ],
        rgbCvdFromRgb_2: [
            0.37009, 0.8854, -0.25549, 0.25767, 0.63782, 0.10451, -0.0195,
            0.02741, 0.99209,
        ],
        separationPlaneNormal: [-0.00293, -0.00645, 0.00938],
    },

    tritan: {
        rgbCvdFromRgb_1: [
            1.01354, 0.14268, -0.15622, -0.01181, 0.87561, 0.13619, 0.07707,
            0.81208, 0.11085,
        ],
        rgbCvdFromRgb_2: [
            0.93337, 0.19999, -0.13336, 0.05809, 0.82565, 0.11626, -0.37923,
            1.13825, 0.24098,
        ],
        separationPlaneNormal: [0.0396, -0.02831, -0.01129],
    },
};

function brettel(srgb, t, severity) {
    // Go from sRGB to linearRGB
    var rgb = Array(3);
    rgb[0] = sRGB_to_linearRGB_Lookup[srgb[0]];
    rgb[1] = sRGB_to_linearRGB_Lookup[srgb[1]];
    rgb[2] = sRGB_to_linearRGB_Lookup[srgb[2]];

    var params = brettel_params[t];
    var separationPlaneNormal = params["separationPlaneNormal"];
    var rgbCvdFromRgb_1 = params["rgbCvdFromRgb_1"];
    var rgbCvdFromRgb_2 = params["rgbCvdFromRgb_2"];

    // Check on which plane we should project by comparing wih the separation plane normal.
    var dotWithSepPlane =
        rgb[0] * separationPlaneNormal[0] +
        rgb[1] * separationPlaneNormal[1] +
        rgb[2] * separationPlaneNormal[2];
    var rgbCvdFromRgb =
        dotWithSepPlane >= 0 ? rgbCvdFromRgb_1 : rgbCvdFromRgb_2;

    // Transform to the full dichromat projection plane.
    var rgb_cvd = Array(3);
    rgb_cvd[0] =
        rgbCvdFromRgb[0] * rgb[0] +
        rgbCvdFromRgb[1] * rgb[1] +
        rgbCvdFromRgb[2] * rgb[2];
    rgb_cvd[1] =
        rgbCvdFromRgb[3] * rgb[0] +
        rgbCvdFromRgb[4] * rgb[1] +
        rgbCvdFromRgb[5] * rgb[2];
    rgb_cvd[2] =
        rgbCvdFromRgb[6] * rgb[0] +
        rgbCvdFromRgb[7] * rgb[1] +
        rgbCvdFromRgb[8] * rgb[2];

    // Apply the severity factor as a linear interpolation.
    // It's the same to do it in the RGB space or in the LMS
    // space since it's a linear transform.
    rgb_cvd[0] = rgb_cvd[0] * severity + rgb[0] * (1.0 - severity);
    rgb_cvd[1] = rgb_cvd[1] * severity + rgb[1] * (1.0 - severity);
    rgb_cvd[2] = rgb_cvd[2] * severity + rgb[2] * (1.0 - severity);

    // Go back to sRGB
    return [
        sRGB_from_linearRGB(rgb_cvd[0]),
        sRGB_from_linearRGB(rgb_cvd[1]),
        sRGB_from_linearRGB(rgb_cvd[2]),
    ];
}

// Adjusted from the hcirn code
function monochrome_with_severity(srgb, severity) {
    var z = Math.round(srgb[0] * 0.299 + srgb[1] * 0.587 + srgb[2] * 0.114);
    var r = z * severity + (1.0 - severity) * srgb[0];
    var g = z * severity + (1.0 - severity) * srgb[1];
    var b = z * severity + (1.0 - severity) * srgb[2];
    return [r, g, b];
}

In [ ]:
// Cost function including weights
const cost = (state) => {
    const energyWeight = 1;
    const rangeWeight = 1;
    const targetWeight = 1;
    const protanopiaWeight = 0.33;
    const deuteranopiaWeight = 0.33;
    const tritanopiaWeight = 0.33;

    const normalDistances = distances(state);
    const protanopiaDistances = distances(state, "Protanopia");
    const deuteranopiaDistances = distances(state, "Deuteranopia");
    const tritanopiaDistances = distances(state, "Tritanopia");

    const energyScore =  100 - average(normalDistances); 
    const protanopiaScore = 100 - average(protanopiaDistances);
    const deuteranopiaScore = 100 - average(deuteranopiaDistances);
    const tritanopiaScore = 100 - average(tritanopiaDistances);
    const rangeScore = range(normalDistances);
    const targetScore = averageDistanceFromtargetColors(state);

    return (
        energyWeight * energyScore +
        targetWeight * targetScore +
        rangeWeight * rangeScore +
        protanopiaWeight * protanopiaScore +
        deuteranopiaWeight * deuteranopiaScore +
        tritanopiaWeight * tritanopiaScore
    );
};

In [ ]:
// the simulated annealing algorithm
const optimize = (n = 5) => {
    const colors = [];
    for (let i = 0; i < n; i++) {
        colors.push(randomColor());
    }

    const startColors = Array.from(colors);
    const startCost = cost(startColors);

    // intialize hyperparameters
    let temperature = 1000;
    const coolingRate = 0.99;
    const cutoff = 0.0001;

    // iteration loop
    while (temperature > cutoff) {
        // for each color
        for (let i = 0; i < colors.length; i++) {
            // copy old colors
            const newColors = colors.map((color) => color);
            // move the current color randomly
            newColors[i] = randomNearbyColor(newColors[i]);
            // choose between the current state and the new state
            // based on the difference between the two, the temperature
            // of the algorithm, and some random chance
            const delta = cost(newColors) - cost(colors);
            const probability = Math.exp(-delta / temperature);
            if (Math.random() < probability) {
                colors[i] = newColors[i];
            }
        }
        console.log(`Current cost: ${cost(colors)}`);

        // decrease temperature
        temperature *= coolingRate;
    }

    console.log(`
Start colors: ${startColors.map((color) => color.hex())}
Start cost: ${startCost}
Final colors: ${colors.map((color) => color.hex())}
Final cost: ${cost(colors)}
Cost difference: ${cost(colors) - startCost}`);
    return colors;
};

In [ ]:
# Generate a color scheme with 8 colors.
optimize(8);